## Prepare settings

In [ ]:
# add path
import os
os.chdir("/content/drive/MyDrive/Code/LMF_attention_tt/ieamocap")
import sys;
sys.path.append('/content/drive/MyDrive/Code/LMF_attention_tt/tensor_fusion')
sys.path.insert(0, '..')

from model import LMF, SubNet
from utils import load_iemocap
from tensor_fusion.module import AdaptiveRankLinear
from tensor_layers.Transformer_tensor import TextSubNet_attention
from tensor_layers.layers import TensorizedLinear, TensorizedLinear_module

# Enable this and writer ovject to track metrics during training
from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter(comment=comment, log_dir='./tt_runs')

from torch.utils.data import DataLoader
from torch.autograd import Variable


from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
import os
import argparse
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import csv

DTYPE = torch.FloatTensor
LONG = torch.LongTensor

def display(f1_score, accuracy_score):
    print("F1-score on test set is {}".format(f1_score))
    print("Accuracy score on test set is {}".format(accuracy_score))

data_path = '/content/drive/MyDrive/data/'

# emotion = 'neutral'
output_dim = 2

# Training parameters
epochs = 125
# epochs = 5
model_path_dir = './tt_models/'
signiture = 'lmf'
patience = 50

factor_lr = 0.003
lr = 0.0005

r = 8
batch_sz = 64
decay = 0.001
cuda = 1

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dtype = torch.float32

complete = True

curr_patience = patience

/usr/local/lib/python3.8/site-packages/tltorch/factorized_tensors/core.py:145: UserWarning: Creating a subclass of FactorizedTensor TensorizedTensor with no name.
  warnings.warn(f'Creating a subclass of FactorizedTensor {cls.__name__} with no name.')


## Model Structure Test

In [ ]:
# Alex
adr = 0.3
vdr = 0.1
tdr = 0.15

ahid = 32
vhid = 32
thid = 64
audio_out = ahid
video_out = vhid
text_out = thid
n_layers = 2
n_head   = 3

emotion = 'happy'
train_set, valid_set, test_set, data_dims = load_iemocap(data_path, emotion)
train_iterator = DataLoader(train_set, batch_size=batch_sz, num_workers=2, shuffle=True)
valid_iterator = DataLoader(valid_set, batch_size=batch_sz, num_workers=2, shuffle=True)
test_iterator = DataLoader(test_set, batch_size=batch_sz, num_workers=2, shuffle=True)

input_dims = [80,36,300,20]
# def __init__(self, input_dims, hidden_dims, audio_out, video_out, text_out, n_layers, n_head, dropouts, output_dim, rank, max_rank, TT_FUSION, use_LSTM, tensor_type, device, dtype, use_softmax=False)
model = LMF(input_dims, (ahid, vhid, thid), audio_out, video_out, text_out, n_layers, n_head, (adr, vdr, tdr, 0.5), output_dim, 6, 4, 0, 1, 'TTM',device, dtype)
model

Audio feature dimension is: 74
Visual feature dimension is: 35
Text feature sequence length is: 20
Text feature dimension is: 300


TypeError: ignored

In [ ]:
x = next(iter(train_iterator))
x_a = Variable(x[0].float().type(DTYPE), requires_grad=False)
x_v = Variable(x[1].float().type(DTYPE), requires_grad=False)
x_t = Variable(x[2].float().type(DTYPE), requires_grad=False)
y = Variable(x[-1].view(-1, output_dim).float().type(DTYPE), requires_grad=False)

print(x_a[0])
print(x_v[0])
print(x_t[0])
print(y[0])

## BP Training Loop

In [ ]:
# trianing setting (execute if only find best)
# rank_list = [3,4,5,6]

signiture = 'attn'

adr = 0.3
vdr = 0.1
tdr = 0.15
fdr = 0.5
dropouts = (adr, vdr, tdr, fdr)

ahid = 32
vhid = 32
thid = 64
ahid_list = [8,16,32]
vhid_list = [4,8,16]
thid_list = [128,256,64]

audio_out = ahid
video_out = vhid
text_out = thid
text_out_list = [7,15,23,31]

output_dim = 2

max_rank = 4
max_rank_list = [2,4,6,8,10]

fusion_rank = 8
fusion_rank_list = [4,8]

n_layers = 1
n_layers_list = [1,2,3]
n_layers_list = [1,2]

n_head = 2
n_head_list = [2,3,4,5,6]
n_head_list = [2,3]

TT_FUSION = 0
TT_FUSION_list = [1, 0]

use_LSTM = 0
use_LSTM_list = [1, 0]

lr = 0.001
lr_list = [0.001, 0.003]
# lr_list = [0.0003, 0.0005, 0.001, 0.003]

decay = 0.001
decay_list = [0, 0.002, 0.01]
# decay_list = [0, 0.001, 0.002, 0.01]

batch_sz = 64
batch_sz_list = [16, 32, 64]

emotion = 'happy'
emotion_list = ['happy','sad','angry','neutral']

train_set, valid_set, test_set, data_dims = load_iemocap(data_path, emotion)
train_iterator = DataLoader(train_set, batch_size=batch_sz, num_workers=2, shuffle=True)
valid_iterator = DataLoader(valid_set, batch_size=batch_sz, num_workers=2, shuffle=True)
test_iterator = DataLoader(test_set, batch_size=batch_sz, num_workers=2, shuffle=True)

print("Train batch:", len(train_iterator))
print("Valid batch:", len(valid_iterator))
print("Test batch:", len(test_iterator))

input_dims = [80,36,300,20]

runs = 2

Audio feature dimension is: 74
Visual feature dimension is: 35
Text feature sequence length is: 20
Text feature dimension is: 300
Train batch: 43
Valid batch: 13
Test batch: 15


### Text classifier mdoel test

In [ ]:
######################### Test classifier model
text_in = input_dims[2]
text_seq = input_dims[3]

d_qkv = text_in//n_head
d_model = text_in
d_inner = text_in

model = TextSubNet_attention(
      n_src_vocab=input_dims[3], d_word_vec=input_dims[2], n_layers=n_layers, n_head=n_head, d_q=d_qkv, d_k=d_qkv, d_v=d_qkv,
      d_model=d_model, d_inner=d_inner, pad_idx=None, dropout=tdr, n_position=text_seq, scale_emb=False,
      emb_shape = [[16,16,8,5],[4,4,8,4]],emb_rank = 16, emb_tensor_type = 'TensorTrainMatrix',
      attention_shape = [[[5,4,3,5],[5,4,3,5]],[[5,4,3,5],[5,4,3,5]]], attention_rank = [max_rank,max_rank],attention_tensor_type = 'TensorTrainMatrix',
      ffn_shape = [[[5,4,3,5],[5,4,3,5]],[[5,4,3,5],[5,4,3,5]]], ffn_rank = [max_rank,max_rank],ffn_tensor_type = 'TensorTrainMatrix',
      d_classifier=d_inner, num_class = output_dim, dropout_classifier = 0.2,
      classifier_shape = [[5,4,3,5],[1,2,1,1]],classifier_rank = max_rank,classifier_tensor_type = 'TensorTrainMatrix',
      bit_attn = 8, scale_attn = 2**(-5),
      bit_ffn = 8, scale_ffn = 2**(-5),
      bit_a = 8, scale_a = 2**(-5),
      quantized = False,
      tensorized=True)

if cuda == 1:
    model = model.cuda()
    DTYPE = torch.cuda.FloatTensor
    LONG = torch.cuda.LongTensor

x = next(iter(train_iterator))
x_a = torch.cat((Variable(x[0].float().type(DTYPE), requires_grad=False), Variable(torch.zeros(batch_sz,(input_dims[0]-data_dims[0])).type(DTYPE), requires_grad=False)), dim=1)
x_v = torch.cat((Variable(x[1].float().type(DTYPE), requires_grad=False), Variable(torch.zeros(batch_sz,(input_dims[1]-data_dims[1])).type(DTYPE), requires_grad=False)), dim=1)
x_t = Variable(x[2].float().type(DTYPE), requires_grad=False)
y = Variable(x[-1].view(-1, output_dim).float().type(DTYPE), requires_grad=False)

src_mask = None
output   = model(x_t, src_mask, return_attns=False)

loss = criterion(output, y)

for name, layer in model.named_modules():
  print(name, layer)

'''
print(x_a.size())
print(x_v.size())
print(x_t.size())

print(x_a)
print(x_v)
print(x_t)
print(y)
print(output)
print(loss)

output = output.cpu().data.numpy().reshape(-1, output_dim)
y = y.cpu().data.numpy().reshape(-1, output_dim)

all_true_label = np.argmax(y,axis=1)
all_predicted_label = np.argmax(output,axis=1)

print(all_true_label)
print(all_predicted_label)

f1 = f1_score(all_true_label, all_predicted_label, average='weighted')
acc_score = accuracy_score(all_true_label, all_predicted_label)

print(f1)
print(acc_score)
'''

## BP fusion model training

In [ ]:
# for max_rank in rank_list:
'''
for fusion_rank in fusion_rank_list:
  for max_rank in max_rank_list:
    for run in range(runs):
        model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}.pt".format(signiture, emotion,fusion_rank,max_rank,run))
'''
'''
for max_rank in max_rank_list:
  for fusion_rank in fusion_rank_list:
    for n_layers in n_layers_list:
      for n_head in n_head_list:
'''
'''
for emotion in emotion_list:
  train_set, valid_set, test_set, input_dims = load_iemocap(data_path, emotion)
  train_iterator = DataLoader(train_set, batch_size=batch_sz, num_workers=2, shuffle=True)
  valid_iterator = DataLoader(valid_set, batch_size=len(valid_set), num_workers=2, shuffle=True)
  test_iterator = DataLoader(test_set, batch_size=len(test_set), num_workers=2, shuffle=True)

'''
'''
for lr in lr_list:
  for decay in decay_list:
    for run in range(runs):
      model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,lr,decay,run))
'''
'''
for TT_FUSION in TT_FUSION_list:
  if TT_FUSION == 0:
    signiture = 'attn_0'
    lr = 0.0005
    decay = 0.01
  else:
    signiture = 'attn_1'
    lr = 0.003
    decay = 0.002
  for max_rank in max_rank_list:
    for fusion_rank in fusion_rank_list:
      for run in range(runs):
        model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,n_layers,n_head,run))
'''
for use_LSTM in use_LSTM_list:
  if use_LSTM == 0:
    signiture = 'LSTM_0'
  else:
    signiture = 'ATTN_0'
  for max_rank in max_rank_list:
    for fusion_rank in fusion_rank_list:
      for run in range(runs):
        model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,n_layers,n_head,run))
        print("Temp location for models: {}".format(model_path))
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        best_valid_acc = 0
        best_valid_loss = float('inf')
        ######################### Fusion Model
        model = LMF(input_dims, (ahid, vhid, thid), audio_out, video_out, text_out, n_layers, n_head, dropouts, output_dim, fusion_rank, max_rank, TT_FUSION, use_LSTM, 'TTM',device, dtype)

        ######################### Test classifier model
        text_in = input_dims[2]
        text_seq = input_dims[3]

        d_qkv = text_in//n_head
        d_model = text_in
        d_inner = text_in

        text_prob = dropouts[2]
        model = TextSubNet_attention(
              n_src_vocab=input_dims[3], d_word_vec=input_dims[2], n_layers=n_layers, n_head=n_head, d_q=d_qkv, d_k=d_qkv, d_v=d_qkv,
              d_model=d_model, d_inner=d_inner, pad_idx=None, dropout=text_prob, n_position=text_seq, scale_emb=False,
              emb_shape = [[16,16,8,5],[4,4,8,4]],emb_rank = 16, emb_tensor_type = 'TensorTrainMatrix',
              attention_shape = [[[5,4,3,5],[5,4,3,5]],[[5,4,3,5],[5,4,3,5]]], attention_rank = [max_rank,max_rank],attention_tensor_type = 'TensorTrainMatrix',
              ffn_shape = [[[5,4,3,5],[5,4,3,5]],[[5,4,3,5],[5,4,3,5]]], ffn_rank = [max_rank,max_rank],ffn_tensor_type = 'TensorTrainMatrix',
              d_classifier=d_inner, num_class = output_dim, dropout_classifier = 0.2,
              classifier_shape = [[5,4,3,5],[5,4,3,5]],classifier_rank = max_rank,classifier_tensor_type = 'TensorTrainMatrix',
              bit_attn = 8, scale_attn = 2**(-5),
              bit_ffn = 8, scale_ffn = 2**(-5),
              bit_a = 8, scale_a = 2**(-5),
              quantized = False,
              tensorized=True)

        ######################### Continue
        if cuda == 1:
            model = model.cuda()
            DTYPE = torch.cuda.FloatTensor
            LONG = torch.cuda.LongTensor

        # criterion = nn.CrossEntropyLoss(reduction='sum')
        # factors = list(model.parameters())[:3]
        # other = list(model.parameters())[3:]
        # optimizer = optim.Adam([{"params": factors, "lr": factor_lr}, {"params": other, "lr": lr}], weight_decay=decay)

        # criterion = nn.BCEWithLogitsLoss(reduction='sum')
        criterion = nn.CrossEntropyLoss(reduction='sum')
        optimizer = optim.Adam(list(model.parameters()), lr, weight_decay=decay)

        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)    # 3

        comment = f'rank = {max_rank} run = {run}'
        writer = SummaryWriter(comment=comment, log_dir='./tt_runs')

        for e in range(epochs):
            model.train()
            model.zero_grad()
            avg_train_loss = 0.0
            for batch in train_iterator:
                model.zero_grad()

                x = batch[:-1]
                x_a = Variable(x[0].float().type(DTYPE), requires_grad=False)
                x_v = Variable(x[1].float().type(DTYPE), requires_grad=False)
                x_t = Variable(x[2].float().type(DTYPE), requires_grad=False)
                y = Variable(batch[-1].view(-1, output_dim).float().type(DTYPE), requires_grad=False)
                try:
                    output = model(x_a, x_v, x_t)
                except ValueError as e:
                    print(x_a.data.shape)
                    print(x_v.data.shape)
                    print(x_t.data.shape)
                    raise e
                loss = criterion(output, y)
                loss.backward()
                avg_train_loss += loss.data.item()
                optimizer.step()

            avg_train_loss = avg_train_loss / len(train_set)
            print("Epoch {} complete! Average Training loss: {}".format(e, avg_train_loss))
            writer.add_scalar("Avg Train Loss", avg_train_loss, e)

            # Terminate the training process if run into NaN
            if np.isnan(avg_train_loss):
                print("Training got into NaN values...\n\n")
                complete = False
                break

            model.eval()
            avg_valid_loss = 0.0
            for batch in valid_iterator:
                x = batch[:-1]
                x_a = Variable(x[0].float().type(DTYPE), requires_grad=False)
                x_v = Variable(x[1].float().type(DTYPE), requires_grad=False)
                x_t = Variable(x[2].float().type(DTYPE), requires_grad=False)
                y = Variable(batch[-1].view(-1, output_dim).float().type(DTYPE), requires_grad=False)
                output = model(x_a, x_v, x_t)
                valid_loss = criterion(output, y)
                avg_valid_loss += valid_loss.data.item()

            if np.isnan(avg_valid_loss):
                print("Training got into NaN values...\n\n")
                complete = False
                break

            avg_valid_loss = avg_valid_loss / len(valid_set)
            print("Validation loss is: {}".format(avg_valid_loss))
            writer.add_scalar("Avg Valid Loss", avg_valid_loss, e)

            all_true_label = np.argmax( y.cpu().data.numpy().reshape(-1, output_dim),axis=1)
            all_predicted_label = np.argmax(output.cpu().data.numpy().reshape(-1, output_dim),axis=1)

            acc_score = accuracy_score(all_true_label, all_predicted_label)


            if (avg_valid_loss < best_valid_loss):
                curr_patience = patience
                best_valid_loss = avg_valid_loss
                torch.save(model, model_path)
                print("Found new best model, saving to disk...")
            else:
                curr_patience -= 1

            if curr_patience <= 0:
                break



#         ------------Code to Write Best Metrics(F1/Acc) out of Run --------------
#         best_model = torch.load(model_path)
#         best_model.eval()
#         best_model.to(device)

#         for batch in test_iterator:
#             x = batch[:-1]
#             x_a = Variable(x[0].float().type(DTYPE), requires_grad=False)
#             x_v = Variable(x[1].float().type(DTYPE), requires_grad=False)
#             x_t = Variable(x[2].float().type(DTYPE), requires_grad=False)
#             y = Variable(batch[-1].view(-1, output_dim).float().type(DTYPE), requires_grad=False)
#             output_test = best_model(x_a, x_v, x_t)

#         output_test = output_test.cpu().data.numpy().reshape(-1, output_dim)
#         y = y.cpu().data.numpy().reshape(-1, output_dim)

#         # these are the needed metrics
#         all_true_label = np.argmax(y,axis=1)
#         all_predicted_label = np.argmax(output_test,axis=1)

#         f1 = f1_score(all_true_label, all_predicted_label, average='weighted')
#         acc_score = accuracy_score(all_true_label, all_predicted_label)


#         writer.add_hparams(
#             {"max_rank": max_rank, "run": run},
#             {
#                 'hparam/accuracy': acc_score, 'hparam/f1': f1
#             }
#         )
# writer.close()

## Eval Best Model

In [ ]:
DTYPE = torch.cuda.FloatTensor
def eval_at_model_path(model_path, test_iterator):
    best_model = torch.load(model_path)
    best_model.eval()
    best_model.to(device)

    for batch in test_iterator:
        x = batch[:-1]
        x_a = Variable(x[0].float().type(DTYPE), requires_grad=False)
        x_v = Variable(x[1].float().type(DTYPE), requires_grad=False)
        x_t = Variable(x[2].float().type(DTYPE), requires_grad=False)
        y = Variable(batch[-1].view(-1, output_dim).float().type(torch.cuda.LongTensor), requires_grad=False)
        # old
        output_test = best_model(x_a, x_v, x_t)
        # new
        # X = [x_a, x_v, x_t]
        # output_test = best_model(X)


    output_test = output_test.cpu().data.numpy().reshape(-1, output_dim)
    y = y.cpu().data.numpy().reshape(-1, output_dim)


    # these are the needed metrics
    all_true_label = np.argmax(y,axis=1)
    all_predicted_label = np.argmax(output_test,axis=1)

    f1 = f1_score(all_true_label, all_predicted_label, average='weighted')
    acc_score = accuracy_score(all_true_label, all_predicted_label)

    display(f1, acc_score)

    return acc_score
    # print("LSTM HH Max Rank", model.text_subnet.rnn.layer_hh.weight_tensor.estimate_rank(), model.text_subnet.rnn.layer_hh.weight_tensor.tensor_type)
    # print("LSTM IH Max Rank", model.text_subnet.rnn.layer_ih.weight_tensor.estimate_rank(),)

## ZO-text attn training

In [ ]:
# ZO_SGD_estimate
import math
import copy
pi = torch.tensor(math.pi)

def perturb_bernoulli(weight, sigma):
  mask = 0.5*torch.ones_like(weight)
  mask = -sigma*torch.ones_like(weight) + 2*sigma*torch.bernoulli(mask)

  return mask

def perturb_gaussian(weight, sigma):
  w_size = weight.size()

  return torch.normal(0, sigma, size=w_size)

def ZO_SGD_estimate(model, criterion, data, target, sigma, K, lr, signSGD=True, sparse=True, symmetric=True) -> list:
  pred = model(data)
  loss = criterion(pred, target)
  # estimate gradient
  G = []
  for k in range(K):
    ##################### f(x+delta_k) start ##########################
    g_model=copy.deepcopy(model)
    for m in g_model.modules():
      if isinstance(m, nn.Linear):
        # Tensorized
        if isinstance(m, (TensorizedLinear, TensorizedLinear_module)):
          # TT-Matrix
          if isinstance(m.tensor, TensorTrainMatrix):
            dims = len(m.tensor.dims[0])
            if sparse == True:
              dim_range = [dims-1]
            else:
              dim_range = range(dims)
            # dims = 1
            m.delta_ = []
            # print(dims)
            # print('not perturbed', m.tensor.factors[0].data[0,0,0,0])
            for dim in dim_range:
              tt_core = m.tensor.factors[dim].data
              delta_ = perturb_gaussian(tt_core, sigma)
              delta_ = delta_.to(device)
              m.delta_.append(delta_)

              tt_core = tt_core + delta_
              m.tensor.factors[dim] = nn.Parameter(tt_core)

            # print('perturbed', m.tensor.factors[0].data[0,0,0,0])

        # not tensorized
        else:
          weight = m.weight.data
          # print('model', weight[0,0])
          m.delta_ = perturb_gaussian(weight, sigma)
          m.delta_ = m.delta_.to(device)
          m.weight = nn.Parameter(weight + m.delta_)
          # print('x+delta_k',m.weight.data[0,0])

    pred_1 = g_model(data)
    loss_1 = criterion(pred_1, target)
    ##################### f(x+delta_k) end ##########################
    if symmetric == True:
      ##################### f(x-delta_k) start ##########################
      # use the same g_model to copy its m.delta_
      for m in g_model.modules():
        if isinstance(m, nn.Linear):
          # Tensorized
          if isinstance(m, (TensorizedLinear, TensorizedLinear_module)):
            # TT-Matrix
            if isinstance(m.tensor, TensorTrainMatrix):
              dims = len(m.tensor.dims[0])
              # dims = 1
              # print(dims)
              # print('not perturbed', m.tensor.factors[0].data[0,0,0,0])
              if sparse == True:
                dim_range = [dims-1]
              else:
                dim_range = range(dims)
              for dim in dim_range:
                tt_core = m.tensor.factors[dim].data
                # delta_ = m.delta_[dim]
                delta_ = m.delta_.pop(0)

                tt_core = tt_core - 2 * delta_
                m.tensor.factors[dim] = nn.Parameter(tt_core)
                m.delta_.append(delta_)
              # print('perturbed', m.tensor.factors[0].data[0,0,0,0])

          # not tensorized
          else:
            weight = m.weight.data
            # print('model',weight[0,0])
            m.weight = nn.Parameter(weight - 2 * m.delta_)
            # print('x-delta_k',m.weight.data[0,0])

      pred_2 = g_model(data)
      loss_2 = criterion(pred_2, target)

      # accumulate gradients
      for m in g_model.modules():
        if isinstance(m, nn.Linear):
          # Tensorized
          if isinstance(m, (TensorizedLinear, TensorizedLinear_module)):
            # TT-Matrix
            if isinstance(m.tensor, TensorTrainMatrix):
              dims = len(m.tensor.dims[0])
              if sparse == True:
                dim_range = [dims-1]
              else:
                dim_range = range(dims)
              # dims = 1
              if k == 0:
                g_ = []
                for dim in dim_range:
                  # delta_ = m.delta_[dim]
                  delta_ = m.delta_.pop(0)
                  g = (loss_1-loss_2)/(2*sigma**2)/K * delta_
                  g_.append(g)
                  m.delta_.append(delta_)

                G.append(g_)
              else:
                g_ = G.pop(0)
                for dim in dim_range:
                  # delta_ = m.delta_[dim]
                  delta_ = m.delta_.pop(0)
                  g = g_.pop(0)
                  g = g + (loss_1-loss_2)/(2*sigma**2)/K * delta_
                  g_.append(g)
                  m.delta_.append(delta_)

                G.append(g_)
              # print('g_=',g_[0][0,0,0,0])
          # Not Tensorized
          else:
            delta_ = m.delta_
            if k == 0:
              g = (loss_1-loss_2)/(2*sigma**2)/K * delta_
              G.append(g)
            else:
              g = G.pop(0)
              g = g + (loss_1-loss_2)/(2*sigma**2)/K * delta_
              G.append(g)

      ##################### f(x-delta_k) end ##########################
    else:
      ##################### f(x) ##########################
      for m in g_model.modules():
        if isinstance(m, nn.Linear):
          # Tensorized
          if isinstance(m, (TensorizedLinear, TensorizedLinear_module)):
            # TT-Matrix
            if isinstance(m.tensor, TensorTrainMatrix):
              dims = len(m.tensor.dims[0])
              if sparse == True:
                dim_range = [dims-1]
              else:
                dim_range = range(dims)
              # dims = 1
              if k == 0:
                g_ = []
                for dim in dim_range:
                  # delta_ = m.delta_[dim]
                  delta_ = m.delta_.pop(0)
                  g = (loss_1-loss)/(sigma**2)/K * delta_
                  g_.append(g)
                  m.delta_.append(delta_)

                G.append(g_)
              else:
                g_ = G.pop(0)
                for dim in dim_range:
                  # delta_ = m.delta_[dim]
                  delta_ = m.delta_.pop(0)
                  g = g_.pop(0)
                  g = g + (loss_1-loss)/(sigma**2)/K * delta_
                  g_.append(g)
                  m.delta_.append(delta_)

                G.append(g_)
              # print('g_=',g_[0][0,0,0,0])
          # Not Tensorized
          else:
            delta_ = m.delta_
            if k == 0:
              g = (loss_1-loss)/(sigma**2)/K * delta_
              G.append(g)
            else:
              g = G.pop(0)
              g = g + (loss_1-loss)/(sigma**2)/K * delta_
              G.append(g)
  # estimate grad end
  ############################### SGD ################################
  for m in model.modules():
    if isinstance(m, nn.Linear):
      # tensorized
      if isinstance(m, (TensorizedLinear, TensorizedLinear_module)):
        if isinstance(m.tensor, TensorTrainMatrix):
          dims = len(m.tensor.dims[0])
          if sparse == True:
            dim_range = [dims-1]
          else:
            dim_range = range(dims)
          # dims = 1
          g_ = G.pop(0)
          # print('SGD g_=',g_[0][0,0,0,0])
          for dim in dim_range:
            g = g_.pop(0)
            if signSGD == True:
              # print('tt-core', m.tensor.factors[dim])
              # print('grad', g)
              m.tensor.factors[dim] = nn.Parameter(m.tensor.factors[dim] - lr*torch.sign(g))
            else:
              m.tensor.factors[dim] = nn.Parameter(m.tensor.factors[dim] - lr*g)

          # print('updated', m.tensor.factors[0].data[0,0,0,0])
      # not tensorized
      else:
        g = G.pop(0)
        m.weight = nn.Parameter(m.weight - lr*g)
  return G


In [ ]:
##################### ZO_SCD_estimate ##########################
def ZO_SCD_estimate(model, criterion, data, target, lr, sparse_a=1, sparse_s=0.05, symmetric=True):
  old_pred = model(data)
  old_loss = criterion(old_pred, target)

  for m in model.modules():
    if isinstance(m, nn.Linear):
      # Tensorized
      if isinstance(m, (TensorizedLinear, TensorizedLinear_module)):
        # TT-Matrix
        if isinstance(m.tensor, TensorTrainMatrix):
          dims = len(m.tensor.dims[0])
          dim_range = range(min(dims,sparse_a))
          # dim_range = [0]
          # print(dims)
          # print('not perturbed', m.tensor.factors[0].data[0,0,0,0])
          for dim in dim_range:
            tt_shape = m.tensor.factors[dim].size()
            tt_core = m.tensor.factors[dim].view(-1)

            mask = sparse_s*torch.ones_like(tt_core)
            mask = torch.bernoulli(mask)

            for idx in range(torch.numel(tt_core)):
              if mask[idx] == 1:
                old_value = tt_core[idx]
                pos_perturbed_value = old_value + lr
                neg_perturbed_value = old_value - lr

                tt_core[idx] = pos_perturbed_value
                m.tensor.factors[dim] = tt_core.reshape(tt_shape)

                new_pred = model(data)
                new_loss = criterion(new_pred, target)

                if new_loss < old_loss:
                  old_loss = new_loss
                else:
                  tt_core[idx] = neg_perturbed_value
                  m.tensor.factors[dim] = tt_core.reshape(tt_shape)
                  # old_pred = model(data)
                  # old_loss = criterion(old_pred, target)
                  new_pred = model(data)
                  new_loss = criterion(new_pred, target)

                  if new_loss < old_loss:
                    old_loss = new_loss
                  else:
                    tt_core[idx] = old_value
                    m.tensor.factors[dim] = tt_core.reshape(tt_shape)
      # not tensorized
      else:
        weight = m.weight.data
        # print('model', weight[0,0])
        m.delta_ = perturb_gaussian(weight, sigma)
        m.delta_ = m.delta_.to(device)
        m.weight = nn.Parameter(weight + m.delta_)
        # print('x+delta_k',m.weight.data[0,0])

## ZO training

In [ ]:
signiture = 'BP_text'
run = 1

SGD_lr = 0.002
SGD_gamma = 0.985
sigma = 0.1
n_sample = 10
K = 10

SCD_lr = 0.0002
SCD_gamma = 0.985
my_sparse_a = 3
my_sparse_s = 0.1

my_sparse_a = 3
my_sparse_s = 0.1


model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,n_layers,n_head,lr, decay, run))
print("Temp location for models: {}".format(model_path))
os.makedirs(os.path.dirname(model_path), exist_ok=True)
best_valid_acc = 0
best_valid_loss = float('inf')
######################### Fusion Model
# model = LMF(input_dims, (ahid, vhid, thid), audio_out, video_out, text_out, n_layers, n_head, dropouts, output_dim, fusion_rank, max_rank, TT_FUSION, use_LSTM, 'TTM',device, dtype)


audio_in = input_dims[0]
video_in = input_dims[1]
text_in = input_dims[2]
text_seq = input_dims[3]

d_qkv = text_in//n_head
d_model = text_in
d_inner = text_in

######################### Audio classifier model
'''
model = SubNet(audio_in, ahid, output_dim, adr,
              shape=[[[2,4,2,5],[4,2,2,2]], [[2,2,2,4],[4,2,2,2]], [[2,2,2,4],[1,2,1,1]]],
              max_rank=max_rank, tensor_type='TensorTrainMatrix', device=device, dtype=dtype)
'''
######################### Video classifier model
'''
model = SubNet(video_in, vhid, output_dim, vdr,
              shape=[[[3,2,2,3],[4,2,2,2]], [[2,2,2,4],[4,2,2,2]], [[2,2,2,4],[1,2,1,1]]],
              max_rank=max_rank, tensor_type=tensor_type, device=device, dtype=dtype)
'''

######################### Test classifier model

model = TextSubNet_attention(
      n_src_vocab=input_dims[3], d_word_vec=input_dims[2], n_layers=n_layers, n_head=n_head, d_q=d_qkv, d_k=d_qkv, d_v=d_qkv,
      d_model=d_model, d_inner=d_inner, pad_idx=None, dropout=tdr, n_position=text_seq, scale_emb=False,
      emb_shape = [[16,16,8,5],[4,4,8,4]],emb_rank = 16, emb_tensor_type = 'TensorTrainMatrix',
      attention_shape = [[[5,4,3,5],[5,4,3,5]],[[5,4,3,5],[5,4,3,5]]], attention_rank = [max_rank,max_rank],attention_tensor_type = 'TensorTrainMatrix',
      ffn_shape = [[[5,4,3,5],[5,4,3,5]],[[5,4,3,5],[5,4,3,5]]], ffn_rank = [max_rank,max_rank],ffn_tensor_type = 'TensorTrainMatrix',
      d_classifier=d_inner, num_class = output_dim, dropout_classifier = 0.2,
      classifier_shape = [[5,4,3,5],[1,2,1,1]],classifier_rank = max_rank,classifier_tensor_type = 'TensorTrainMatrix',
      bit_attn = 8, scale_attn = 2**(-5),
      bit_ffn = 8, scale_ffn = 2**(-5),
      bit_a = 8, scale_a = 2**(-5),
      quantized = False,
      tensorized=True)


######################### Continue
if cuda == 1:
    model = model.cuda()
    DTYPE = torch.cuda.FloatTensor
    LONG = torch.cuda.LongTensor

# criterion = nn.CrossEntropyLoss(reduction='sum')
# factors = list(model.parameters())[:3]
# other = list(model.parameters())[3:]
# optimizer = optim.Adam([{"params": factors, "lr": factor_lr}, {"params": other, "lr": lr}], weight_decay=decay)

# criterion = nn.BCEWithLogitsLoss(reduction='sum')
criterion = nn.CrossEntropyLoss(reduction='mean')

optimizer = optim.Adam(list(model.parameters()), lr, weight_decay=decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)    # 3

comment = f'rank = {max_rank} run = {run}'
writer = SummaryWriter(comment=comment, log_dir='./tt_runs')

train_len = len(train_iterator.dataset)
train_batch = len(train_iterator)

valid_len = len(valid_iterator.dataset)
valid_batch = len(valid_iterator)

for e in range(epochs):
    ########################## training ############################
    model.train()
    model.zero_grad()
    avg_train_loss = 0.0
    batch_cnt = 0
    for batch in train_iterator:
        batch_cnt += 1
        model.zero_grad()

        x = batch[:-1]
        batch_rows = x[0].data.shape[0]
        x_a = torch.cat((Variable(x[0].float().type(DTYPE), requires_grad=False), Variable(torch.zeros(batch_rows,(input_dims[0]-data_dims[0])).type(DTYPE), requires_grad=False)), dim=1)
        x_v = torch.cat((Variable(x[1].float().type(DTYPE), requires_grad=False), Variable(torch.zeros(batch_rows,(input_dims[1]-data_dims[1])).type(DTYPE), requires_grad=False)), dim=1)
        x_t = Variable(x[2].float().type(DTYPE), requires_grad=False)
        y = Variable(batch[-1].view(-1, output_dim).float().type(DTYPE), requires_grad=False)

        ########################## BP ############################
        '''
        # output = model(x_a)
        # output = model(x_v)
        output = model(x_t)
        # X = [x_a, x_v, x_t]
        # output = model(X)
        loss = criterion(output, y)
        avg_train_loss += loss.data.item()

        loss.backward()
        optimizer.step()
        '''

        ########################## ZO ############################
        with torch.no_grad():
          # output = model(x_a)
          # output = model(x_v)
          output = model(x_t)
          # X = [x_a, x_v, x_t]
          # output = model(X)
          loss = criterion(output, y)

          if e < 15:
            ZO_SGD_estimate(model, criterion, x_t, y, sigma, K, SGD_lr, signSGD=True, sparse=False, symmetric=True)
          else:
            ZO_SCD_estimate(model, criterion, x_t, y, lr, sparse_a=my_sparse_a, sparse_s=my_sparse_s, symmetric=True)
            # ZO_SGD_estimate(model, criterion, x_t, y, sigma, K, lr, signSGD=False, sparse=True, symmetric=True)

        avg_train_loss += loss.data.item()

        if batch_cnt % 10 == 0:
          # print("epoch {} batch {} SGD Train loss is: {}".format(e, batch_cnt, loss.data.item()))
          writer.add_scalar("SGD Train loss", loss.data.item(), batch_cnt)

    if e % 10 == 9:
        SGD_lr = SGD_lr * SGD_gamma

    avg_train_loss = avg_train_loss / train_batch
    print("\n Epoch {} complete! Average Training loss: {}".format(e, avg_train_loss))
    writer.add_scalar("Avg Train Loss", avg_train_loss, e)

    # Terminate the training process if run into NaN
    if np.isnan(avg_train_loss):
        print("Training got into NaN values...\n\n")
        complete = False
        break

    ########################## Eval ############################
    model.eval()
    avg_valid_loss = 0.0
    f1 = 0.0
    acc_score = 0.0
    for batch in valid_iterator:
        x = batch[:-1]
        batch_rows = x[0].data.shape[0]
        x_a = torch.cat((Variable(x[0].float().type(DTYPE), requires_grad=False), Variable(torch.zeros(batch_rows,(input_dims[0]-data_dims[0])).type(DTYPE), requires_grad=False)), dim=1)
        x_v = torch.cat((Variable(x[1].float().type(DTYPE), requires_grad=False), Variable(torch.zeros(batch_rows,(input_dims[1]-data_dims[1])).type(DTYPE), requires_grad=False)), dim=1)
        x_t = Variable(x[2].float().type(DTYPE), requires_grad=False)
        y = Variable(batch[-1].view(-1, output_dim).float().type(DTYPE), requires_grad=False)

        src_mask = None
        # output = model(x_a)
        # output = model(x_v)
        output = model(x_t)
        # X = [x_a, x_v, x_t]
        # output = model(X)

        valid_loss = criterion(output, y)
        avg_valid_loss += valid_loss.data.item()

        output = output.cpu().data.numpy().reshape(-1, output_dim)
        y = y.cpu().data.numpy().reshape(-1, output_dim)

        # metrics
        all_true_label = np.argmax(y,axis=1)
        all_predicted_label = np.argmax(output,axis=1)

        f1 += f1_score(all_true_label, all_predicted_label, average='weighted')
        acc_score += accuracy_score(all_true_label, all_predicted_label)

    if np.isnan(avg_valid_loss):
        print("Training got into NaN values...\n\n")
        complete = False
        break

    # show avg_valid_loss， acc_score and f1
    avg_valid_loss = avg_valid_loss / valid_batch
    print("Validation loss is: {}".format(avg_valid_loss))
    writer.add_scalar("Avg Valid Loss", avg_valid_loss, e)

    f1 = f1 / valid_batch
    acc_score = acc_score / valid_batch
    print("F1-score on test set is {}".format(f1))
    print("Accuracy score on test set is {}".format(acc_score))


    if (avg_valid_loss < best_valid_loss):
        curr_patience = patience
        best_valid_loss = avg_valid_loss
        torch.save(model, model_path)
        print("Found new best model, saving to disk...")
    else:
        curr_patience -= 1

    '''
    if curr_patience <= 0:
        break
    '''

## ZO Eval

In [ ]:
best_model_path = 0
best_acc = 0
# for max_rank in max_rank_list:

signiture = 'lmf'
model_path_dir = '../history/tt_fusion_layer_head'
emotion_list = ['happy','sad','angry','neutral']
max_rank = 2
fusion_rank = 4
n_layers = 1
n_head = 2
runs = 3

for emotion in emotion_list:
  train_set, valid_set, test_set, input_dims = load_iemocap(data_path, emotion)
  test_iterator = DataLoader(test_set, batch_size=len(test_set), num_workers=2, shuffle=True)
  # for fusion_rank in fusion_rank_list:
  for run in range(runs):
    model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,n_layers,n_head,run))
    print(model_path)
    acc = eval_at_model_path(model_path, test_iterator)
    print("-------------")
    if acc > best_acc:
      best_acc = acc
      best_model_path = model_path
'''
for lr in lr_list:
  for decay in decay_list:
    for run in range(runs):
      model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,lr,decay,run))
'''
'''
for max_rank in max_rank_list:
  for fusion_rank in fusion_rank_list:
    for run in range(runs):
      model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,n_layers,n_head,run))
'''
'''
for TT_FUSION in TT_FUSION_list:
  if TT_FUSION == 0:
    signiture = 'attn_0'
    lr = 0.0005
    decay = 0.01
  else:
    signiture = 'attn_1'
    lr = 0.003
    decay = 0.002
  for max_rank in max_rank_list:
    for fusion_rank in fusion_rank_list:
      for run in range(runs):
        model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,n_layers,n_head,run))
'''
'''
for use_LSTM in use_LSTM_list:
  if use_LSTM == 0:
    signiture = 'LSTM_0'
  else:
    signiture = 'ATTN_0'
  for max_rank in max_rank_list:
    for fusion_rank in fusion_rank_list:
      for run in range(runs):
        model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,n_layers,n_head,run))
        print(model_path)
        acc = eval_at_model_path(model_path, test_iterator)
        print("-------------")
        if acc > best_acc:
            best_acc = acc
            best_model_path = model_path
'''

Audio feature dimension is: 74
Visual feature dimension is: 35
Text feature sequence length is: 20
Text feature dimension is: 300
../history/tt_fusion_layer_head/model_lmf_happy_2_4_1_2_0.pt
F1-score on test set is 0.8223518181720171
Accuracy score on test set is 0.8635394456289979
-------------
../history/tt_fusion_layer_head/model_lmf_happy_2_4_1_2_1.pt
F1-score on test set is 0.8339447188743941
Accuracy score on test set is 0.8678038379530917
-------------
../history/tt_fusion_layer_head/model_lmf_happy_2_4_1_2_2.pt
F1-score on test set is 0.7896951608577283
Accuracy score on test set is 0.8560767590618337
-------------
Audio feature dimension is: 74
Visual feature dimension is: 35
Text feature sequence length is: 20
Text feature dimension is: 300
../history/tt_fusion_layer_head/model_lmf_sad_2_4_1_2_0.pt
F1-score on test set is 0.8275418051577313
Accuracy score on test set is 0.8304904051172708
-------------
../history/tt_fusion_layer_head/model_lmf_sad_2_4_1_2_1.pt
F1-score on tes

'\nfor use_LSTM in use_LSTM_list:\n  if use_LSTM == 0:\n    signiture = \'LSTM_0\'\n  else:\n    signiture = \'ATTN_0\'\n  for max_rank in max_rank_list:\n    for fusion_rank in fusion_rank_list:\n      for run in range(runs):\n        model_path = os.path.join(model_path_dir, "model_{}_{}_{}_{}_{}_{}_{}.pt".format(signiture, emotion,max_rank,fusion_rank,n_layers,n_head,run))\n        print(model_path)\n        acc = eval_at_model_path(model_path, test_iterator)\n        print("-------------")\n        if acc > best_acc:\n            best_acc = acc\n            best_model_path = model_path\n'